# database

> Functions for storing and retrieving contacts

In [ ]:
#| default_exp database

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fasthtml.common import *
from fastcore.test import *
from dataclasses import dataclass
from fastcore.basics import *

from todo.models import *

## Sample Data
 
We'll start with some sample contacts for testing.

In [ ]:
#| export
sample_contacts = [
    Contact(1, "John Doe", "john@example.com", "555-1234", "123 Main St", True),
    Contact(2, "Jane Smith", "jane@example.com", "555-5678", "456 Oak Ave", False),
    Contact(3, "Bob Johnson", "bob@example.com", "555-9012", "789 Pine Rd", False),
]

## Database Operations
 
For simplicity, we'll use an in-memory list as our "database".

In [ ]:
#| export
class ContactDB:
    "Simple in-memory database for contacts"
    def __init__(self, initial_contacts=None):
        "Initialize with optional contacts"
        self.contacts = initial_contacts.copy() if initial_contacts else []
    
    def get_all(self):
        "Get all contacts"
        return self.contacts
    
    def get(self, contact_id):
        "Get a contact by ID"
        return next((c for c in self.contacts if c.id == contact_id), None)
    
    def search(self, query):
        "Search contacts by name, email, or phone"
        if not query:
            return self.contacts
        query = query.lower()
        return [c for c in self.contacts if query in c.name.lower() or 
                                           query in c.email.lower() or 
                                           query in c.phone.lower()]
    
    def add(self, contact):
        "Add a new contact"
        if not self.contacts:
            contact.id = 1
        else:
            contact.id = max(c.id for c in self.contacts) + 1
        self.contacts.append(contact)
        return contact
    
    def update(self, contact):
        "Update an existing contact"
        for i, c in enumerate(self.contacts):
            if c.id == contact.id:
                self.contacts[i] = contact
                return True
        return False
    
    def delete(self, contact_id):
        "Delete a contact by ID"
        original_length = len(self.contacts)
        self.contacts = [c for c in self.contacts if c.id != contact_id]
        return len(self.contacts) < original_length

Let's test our database operations:

In [ ]:
# Initialize DB with sample data
db = ContactDB(sample_contacts)

# Test getting all contacts
test_eq(len(db.get_all()), 3)

# Test getting a single contact
test_eq(db.get(1).name, "John Doe")

# Test search
test_eq(len(db.search("john")), 2)
test_eq(len(db.search("example.com")), 3)

# Test adding a contact
new_contact = Contact(0, "Alice Brown", "alice@example.com", "555-4321", "321 Elm St")
db.add(new_contact)
test_eq(len(db.get_all()), 4)
test_eq(new_contact.id, 4)  # ID should be assigned

# Test updating a contact
updated = Contact(1, "John Updated", "john@example.com", "555-1234", "123 Main St", True)
test_eq(db.update(updated), True)
test_eq(db.get(1).name, "John Updated")

# Test deleting a contact
test_eq(db.delete(2), True)
test_eq(len(db.get_all()), 3)
test_eq(db.get(2), None)

## Default Database Instance
 
We'll create a default database instance with our sample data.

In [ ]:
#| export

# Create a default database instance with sample data
default_db = ContactDB(sample_contacts)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()